In [18]:
import h5py
data = h5py.File("/mnt/petrelfs/zhengjinliang/Data/fold_cloth_air_high_quality/0928_10am_new/episode_53.hdf5", "r")

OSError: Unable to synchronously open file (truncated file: eof = 169476096, sblock->base_addr = 0, stored_eof = 327466532)

In [3]:
from datasets import create_dataloader
data = iter(create_dataloader(
    batch_size = 16, 
  metas_path = "/mnt/petrelfs/zhengjinliang/Data/agiworld/on-site/meta_files", 
  num_actions = 60,
  training = True,
  action_mode = "joint",
  rel_idx = [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13]
))

use action mode: joint
rel_idx: [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13]
== dataset agiworld-on-site-pack-extra with 171 trajs
== dataset agiworld-on-site-restock with 206 trajs
== dataset agiworld-on-site-conveyor with 204 trajs
== dataset agiworld-on-site-cloth with 181 trajs
== dataset agiworld-on-site-pack with 217 trajs
== dataset agiworld-on-site-conveyor-extra with 265 trajs
== dataset agiworld-on-site-pour with 286 trajs
== dataset agiworld-on-site-microwave with 329 trajs


In [12]:
import time
t = time.time()
for _ in range(100):
    next(data)
    print(time.time() - t)
    t = time.time()

0.0009388923645019531
0.00019621849060058594
9.655952453613281e-05
9.512901306152344e-05
8.392333984375e-05
7.486343383789062e-05
7.104873657226562e-05
7.462501525878906e-05
0.577892541885376
0.05624032020568848
0.014762163162231445
7.82012939453125e-05
0.5427501201629639
0.01530599594116211
8.535385131835938e-05
8.273124694824219e-05
0.5575845241546631
0.015039205551147461
7.748603820800781e-05
8.0108642578125e-05
0.5776870250701904
0.015976428985595703
9.131431579589844e-05
8.988380432128906e-05
0.5693707466125488
0.00020456314086914062
7.772445678710938e-05
6.794929504394531e-05
0.546238899230957
0.00014066696166992188
0.0001227855682373047
0.1888294219970703
0.3329036235809326
0.04347729682922363
0.01441192626953125
0.20671343803405762
0.29680800437927246
0.05176210403442383
0.18251585960388184
0.1591649055480957
0.19638442993164062
0.014023542404174805
0.22941017150878906
0.14214324951171875
0.1760396957397461
0.014077901840209961
0.2274177074432373
0.14595842361450195
0.191774368

In [1]:
from mmengine import fileio
import json
path = "/mnt/petrelfs/zhengjinliang/Data/air-agilex/1010_8am_yaxin"

data_list = fileio.list_dir_or_file(path, recursive=True, suffix=".hdf5", list_dir=False)
data_list = [fileio.join_path(path, i) for i in data_list]

with open("/mnt/petrelfs/zhengjinliang/Data/air-agilex/1010_8am_yaxin/meta.json", "w") as f:
    json.dump({
        'dataset_name': "AIR-AGILEX-HQ",
        "language_instruction_key": "language_instruction",
        "observation_key": [
            "observations/images/cam_high",
            "observations/images/cam_left_wrist",
            "observations/images/cam_right_wrist"
        ],
        "datalist": data_list}, f)


In [296]:
from mmengine import fileio
import json
path = "/mnt/petrelfs/zhengjinliang/Data/agiworld/Manipulation-RealRobot/observations/tars/1645"

data_list = []

for sub_path in fileio.list_dir_or_file(path, list_file=False):
    sub_path = fileio.join_path(path, sub_path)
    for ss_path in fileio.list_dir_or_file(sub_path, list_file=False):
        ss_path = fileio.join_path(sub_path, ss_path)
        data_list.append(ss_path)

with open("/mnt/petrelfs/zhengjinliang/Data/agiworld/on-site/meta_files/conveyor-extra.json", "w") as f:
    json.dump({
        'dataset_name': 'agiworld-on-site-conveyor-extra',
        'datalist': data_list
    }, f)

In [302]:
with open("/mnt/petrelfs/zhengjinliang/Data/agiworld/Manipulation-RealRobot/task_info/conveyor_assitant.json", "r") as f:
    ref = json.load(f)
d = {}
num = 0
for i in ref:
    d[i['episode_id']] = True
    for a in i['label_info']['action_config']:
        if 'left' in a['action_text']: 
            d[i['episode_id']]= False
print(num)

1384


In [299]:
new_datalist = []
for i in data_list:
    if d[int(i.split('/')[-1])]: new_datalist.append(i)

In [303]:
len(new_datalist)
with open("/mnt/petrelfs/zhengjinliang/Data/agiworld/on-site/meta_files/conveyor-extra.json", "w") as f:
    json.dump({
        'dataset_name': 'agiworld-on-site-conveyor-extra',
        'datalist': new_datalist
    }, f)

In [66]:

path = "/mnt/petrelfs/zhengjinliang/Data/agiworld/Manipulation-RealRobot/observations/tars/1645"
data_list = []

for sub_path in fileio.list_dir_or_file(path, list_file=False):
    sub_path = fileio.join_path(path, sub_path)
    for ss_path in fileio.list_dir_or_file(sub_path, list_file=False):
        ss_path = fileio.join_path(sub_path, ss_path)
        data_list.append(ss_path)
len(data_list)

171

In [ ]:
# pip install opencv-python tqdm
from pathlib import Path
import av
from tqdm import tqdm
def extract_frames(video_path: str, out_dir: str = "frames"):
    out = Path(out_dir)
    out.mkdir(parents=True, exist_ok=True)

    count = 0
    with av.open(video_path) as container:
        vstream = container.streams.video[0]  # 第一个视频流
        for frame in tqdm(container.decode(vstream)):  # 逐帧解码
            img = frame.to_image()              # 转成 PIL.Image
            img.save(out / f"frame_{count}.jpg", quality=95)
            count += 1
    print(f"Done. Saved {count} frames to '{out_dir}/'")
    
for d in data_list:
    for videos in ['head_color.mp4', 'hand_left_color.mp4', 'hand_right_color.mp4']:
        v_path = fileio.join_path(d, 'videos', videos)
        paths = extract_frames(
            v_path,
            out_dir=v_path[:-4]
        )        

In [32]:
data['action'].keys()

<KeysViewHDF5 ['effector', 'end', 'head', 'joint', 'robot', 'waist']>

In [42]:
data['state']['end'].keys()

<KeysViewHDF5 ['angular', 'orientation', 'position', 'velocity', 'wrench']>

In [19]:
data = iter(create_dataloader(
        batch_size = 16,
        metas_path = "/mnt/petrelfs/zhengjinliang/HeteroDiffusionPolicy/AbsEEFFlowV8/meta_files/finetuning/3-view-dual/flod_cloth_hq.json",
        num_actions= 30,
        training = True
    ))

== dataset AIR-AGILEX-HQ with 1166 trajs


In [ ]:
from transformers import AutoModelForCausalLM
vlm = AutoModelForCausalLM.from_pretrained(
            "microsoft/Florence-2-large",
            torch_dtype="auto",
            trust_remote_code=True
        )

In [12]:
import h5py
f = h5py.File("/mnt/petrelfs/zhengjinliang/Data/agiworld/on-site/tars/pour/1568747-1571186/1568747/aligned_joints.h5", "r")

In [228]:
np.abs(f['action']['joint']['position'][1:] - f['state']['joint']['position'][:-1]).max()

0.14293679471416743

In [244]:
f['state']['joint']['position']

<HDF5 dataset "position": shape (1069, 14), type "<f8">

In [17]:
from datasets.common import open_h5, quat_to_rotate6d
from scipy.interpolate import interp1d
import numpy as np
try:
    gripper_left = f['action']['effector']['position'][:, 0] # [T]
    gripper_right = f['action']['effector']['position'][:, 1] # [T]
except:
    gripper_left = f['action']['left_effector']['position'][:, 0]
    gripper_right = f['action']['right_effector']['position'][:, 0]
    
joints = f['state']['joint']['position'][:] # [T, 14]
xyz_position_left = f['state']['end']['position'][:, 0] # [T, 3]
xyz_position_right = f['state']['end']['position'][:, 1] # [T, 3]
orientation_left = f['state']['end']['orientation'][:, 0] # [T, 4]
orientation_right = f['state']['end']['orientation'][:, 1] # [T, 4]
abs_joint = np.concatenate([joints,
                            gripper_left[:, None],
                            gripper_right[:, None]], axis=-1) # [T, 16]
abs_ee6d = np.concatenate([
    xyz_position_left, quat_to_rotate6d(orientation_left), gripper_left[:, None],
    xyz_position_right, quat_to_rotate6d(orientation_right), gripper_right[:, None]
], axis=-1)
traj_len = len(abs_joint)
skip_num = 0
num_actions = 60
grippers = abs_joint[:, -2:]
chg = np.any(grippers[1:] != grippers[:-1], axis=1)             # chg[k] 表示在行 i=k+1 发生变化
print(np.flatnonzero(chg))
for idx in range(len(abs_joint) - 60):
    if np.abs(abs_ee6d[idx + 1] - abs_ee6d[idx]).max() < 5e-4:
        continue

    # Initial window upper bound
    rel = min(60 + 1, len(abs_joint) - idx)

    # Future nearest gripper change: ensure at least MINCHUNK after it
    right = np.flatnonzero(chg[idx:])  # offsets relative to idx
    if right.size > 0:
        rel = min(right[0] + 10, rel)

    # Past nearest gripper change (use most recent: left[-1])
    left = np.flatnonzero(chg[:idx])
    if left.size > 0:
        rel = min(idx - left[-1] + 10, rel)
    print(idx, rel)


[ 92  93  94 584 585 586]
7 61
8 61
9 61
10 61
11 61
12 61
13 61
14 61
15 61
16 61
17 61
18 61
19 61
20 61
21 61
22 61
23 61
24 61
25 61
26 61
27 61
28 61
29 61
30 61
31 61
32 61
33 61
34 61
35 61
36 61
37 61
38 61
39 61
40 61
41 61
42 60
43 59
44 58
45 57
46 56
47 55
48 54
49 53
50 52
51 51
52 50
53 49
54 48
55 47
56 46
57 45
58 44
59 43
60 42
61 41
62 40
63 39
64 38
65 37
66 36
67 35
68 34
69 33
70 32
71 31
72 30
73 29
74 28
75 27
76 26
77 25
78 24
79 23
80 22
81 21
82 20
83 19
84 18
85 17
86 16
87 15
88 14
89 13
90 12
91 11
92 10
93 10
94 10
95 11
96 12
97 13
98 14
102 18
103 19
104 20
105 21
106 22
107 23
108 24
109 25
110 26
111 27
117 33
118 34
119 35
120 36
121 37
122 38
123 39
124 40
125 41
126 42
127 43
128 44
129 45
130 46
131 47
132 48
133 49
134 50
135 51
136 52
137 53
138 54
139 55
140 56
141 57
142 58
143 59
144 60
145 61
146 61
147 61
148 61
149 61
150 61
151 61
152 61
153 61
154 61
155 61
156 61
157 61
158 61
159 61
160 61
161 61
162 61
163 61
164 61
165 61
166 61
167 6

In [272]:
f['action']['effector']['position'][420:470] 

array([[0.  , 0.  ],
       [0.  , 0.27],
       [0.  , 1.  ],
       [0.  , 1.  ],
       [0.  , 1.  ],
       [0.  , 1.  ],
       [0.  , 1.  ],
       [0.  , 1.  ],
       [0.  , 1.  ],
       [0.  , 1.  ],
       [0.  , 1.  ],
       [0.  , 1.  ],
       [0.  , 1.  ],
       [0.  , 1.  ],
       [0.  , 1.  ],
       [0.  , 1.  ],
       [0.  , 1.  ],
       [0.  , 1.  ],
       [0.  , 1.  ],
       [0.  , 1.  ],
       [0.  , 1.  ],
       [0.  , 1.  ],
       [0.  , 1.  ],
       [0.  , 1.  ],
       [0.  , 1.  ],
       [0.  , 1.  ],
       [0.  , 1.  ],
       [0.  , 1.  ],
       [0.  , 1.  ],
       [0.  , 1.  ],
       [0.  , 1.  ],
       [0.  , 1.  ],
       [0.  , 1.  ],
       [0.  , 1.  ],
       [0.  , 1.  ],
       [0.  , 1.  ],
       [0.  , 1.  ],
       [0.  , 1.  ],
       [0.  , 1.  ],
       [0.  , 1.  ],
       [0.  , 1.  ],
       [0.  , 1.  ],
       [0.  , 1.  ],
       [0.  , 1.  ],
       [0.  , 1.  ],
       [0.  , 1.  ],
       [0.  , 1.  ],
       [0.  ,

In [273]:
(f['state']['effector']['position'][:] > 1.)[420:470]

array([[False, False],
       [False, False],
       [False, False],
       [False,  True],
       [False,  True],
       [False,  True],
       [False,  True],
       [False,  True],
       [False,  True],
       [False,  True],
       [False,  True],
       [False,  True],
       [False,  True],
       [False,  True],
       [False,  True],
       [False,  True],
       [False,  True],
       [False,  True],
       [False,  True],
       [False,  True],
       [False,  True],
       [False,  True],
       [False,  True],
       [False,  True],
       [False,  True],
       [False,  True],
       [False,  True],
       [False,  True],
       [False,  True],
       [False,  True],
       [False,  True],
       [False,  True],
       [False,  True],
       [False,  True],
       [False,  True],
       [False,  True],
       [False,  True],
       [False,  True],
       [False,  True],
       [False,  True],
       [False,  True],
       [False,  True],
       [False,  True],
       [Fal

In [130]:
import numpy as np
from scipy.spatial.transform import Rotation as R

def quat_to_rotate6D(q: np.ndarray) -> np.ndarray:
    return R.from_quat(q).as_matrix()[..., :, :2].reshape(q.shape[:-1] + (6,))
gripper_left = f['action']['effector']['position'][:, 0] # [T]
gripper_right = f['action']['effector']['position'][:, 1] # [T]
joints = f['action']['joint']['position'][:] # [T, 14]
xyz_position_left = f['action']['end']['position'][:, 0] # [T, 3]
xyz_position_right = f['action']['end']['position'][:, 1] # [T, 3]
orientation_left = f['action']['end']['orientation'][:, 0] # [T, 4]
orientation_right = f['action']['end']['orientation'][:, 1] # [T, 4]
abs_joint = np.concatenate([joints, 
                            gripper_left[:, None], 
                            gripper_right[:, None]], axis=-1) # [T, 16]

abs_ee6d = np.concatenate([
    xyz_position_left, quat_to_rotate6D(orientation_left), gripper_left[:, None],
    xyz_position_right, quat_to_rotate6D(orientation_right), gripper_right[:, None]
], axis=-1)

In [132]:
abs_joint.shape

(1336, 16)

In [133]:
abs_ee6d.shape

(1336, 20)

In [137]:
np.abs(abs_joint[1:101, :14] - abs_joint[:100, :14]).sum(-1)

array([1.57326460e-04, 1.57296658e-04, 2.09599733e-04, 1.74582005e-04,
       2.96711922e-04, 2.26825476e-04, 2.26944685e-04, 2.26855278e-04,
       2.61515379e-04, 2.96622515e-04, 2.61664391e-04, 2.09331512e-04,
       2.79188156e-04, 2.79188156e-04, 1.91658735e-04, 3.83973122e-04,
       3.31819057e-04, 2.79217958e-04, 2.96831131e-04, 6.99162483e-05,
       2.61992216e-04, 1.22595563e-02, 1.22243892e-02, 2.55893266e-03,
       2.02642917e-03, 2.09921743e-02, 2.54034105e-02, 2.78757629e-02,
       1.70893783e-02, 1.63297411e-02, 3.76710421e-02, 1.93406479e-02,
       1.98418225e-02, 3.11662270e-02, 3.88345620e-02, 2.59276679e-02,
       4.04812828e-02, 3.31951074e-02, 4.49889775e-02, 3.73256662e-02,
       3.36650466e-02, 3.07311815e-02, 4.21461296e-02, 2.26226073e-02,
       2.61748879e-02, 4.59705566e-02, 3.04289992e-02, 3.60852348e-02,
       2.13485724e-02, 1.93605815e-02, 2.30427966e-02, 3.49150131e-02,
       2.14440174e-02, 2.70499820e-02, 2.93105566e-02, 4.23403092e-02,
      